In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

/home/giyaseddin/miniconda3/envs/text-classification-dtse-challenge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data
train_df = pd.read_csv('../data/processed/clean_train.csv')
valid_df = pd.read_csv('../data/processed/clean_valid.csv')

train_usmpl_df = pd.read_csv('../data/processed/clean_train_upsampled.csv')


# Initialize Sentence Transformer Model
model = SentenceTransformer('BAAI/bge-small-en-v1.5')

# Transform 'clean_content' using Sentence Transformer
X_train = model.encode(train_df['clean_content'].to_list(), show_progress_bar=True)
X_valid = model.encode(valid_df['clean_content'].to_list(), show_progress_bar=True)
X_train_usmpl = model.encode(train_usmpl_df['clean_content'].to_list(), show_progress_bar=True)


# Prepare labels for multilabel classification
y_train = train_df[['cyber_label', 'environmental_issue']]
y_valid = valid_df[['cyber_label', 'environmental_issue']]
y_train_usmpl = train_usmpl_df[['cyber_label', 'environmental_issue']]


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [00:20<00:00,  4.54it/s]


In [3]:

# MultiOutput Classifier (cleaned dataset)
multioutput_classifier = MultiOutputClassifier(SVC(probability=True, random_state=42), n_jobs=-1)
multioutput_classifier.fit(X_train, y_train)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

MultiOutputClassifier(estimator=SVC(probability=True, random_state=42),
                      n_jobs=-1)

In [4]:

# Prediction and evaluation
y_pred = multioutput_classifier.predict(X_valid)
for i, label in enumerate(y_train.columns):
    print(f"Accuracy for {label}: {accuracy_score(y_valid.iloc[:, i], y_pred[:, i])}")
    print(f"Classification Report for {label}:\n", classification_report(y_valid.iloc[:, i], y_pred[:, i]))


Accuracy for cyber_label: 0.9523809523809523
Classification Report for cyber_label:
               precision    recall  f1-score   support

           0       0.96      1.00      0.97       235
           1       0.86      0.35      0.50        17

    accuracy                           0.95       252
   macro avg       0.91      0.67      0.74       252
weighted avg       0.95      0.95      0.94       252

Accuracy for environmental_issue: 0.8888888888888888
Classification Report for environmental_issue:
               precision    recall  f1-score   support

           0       0.89      0.97      0.93       200
           1       0.85      0.56      0.67        52

    accuracy                           0.89       252
   macro avg       0.87      0.77      0.80       252
weighted avg       0.89      0.89      0.88       252



In [5]:

# MultiOutput Classifier (Upsampled cleaned dataset)
multioutput_classifier = MultiOutputClassifier(SVC(probability=True, random_state=42), n_jobs=-1)
multioutput_classifier.fit(X_train_usmpl, y_train_usmpl)


# Prediction and evaluation
y_pred = multioutput_classifier.predict(X_valid)
for i, label in enumerate(y_train_usmpl.columns):
    print(f"Accuracy for {label}: {accuracy_score(y_valid.iloc[:, i], y_pred[:, i])}")
    print(f"Classification Report for {label}:\n", classification_report(y_valid.iloc[:, i], y_pred[:, i]))


Accuracy for cyber_label: 0.9444444444444444
Classification Report for cyber_label:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97       235
           1       0.57      0.71      0.63        17

    accuracy                           0.94       252
   macro avg       0.77      0.83      0.80       252
weighted avg       0.95      0.94      0.95       252

Accuracy for environmental_issue: 0.8690476190476191
Classification Report for environmental_issue:
               precision    recall  f1-score   support

           0       0.94      0.89      0.92       200
           1       0.65      0.79      0.71        52

    accuracy                           0.87       252
   macro avg       0.80      0.84      0.81       252
weighted avg       0.88      0.87      0.87       252



In [6]:
# MultiOutput Classifier
multioutput_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42), n_jobs=-1)
multioutput_classifier.fit(X_train_usmpl, y_train_usmpl)


# Prediction and evaluation
y_pred = multioutput_classifier.predict(X_valid)
for i, label in enumerate(y_train_usmpl.columns):
    print(f"Accuracy for {label}: {accuracy_score(y_valid.iloc[:, i], y_pred[:, i])}")
    print(f"Classification Report for {label}:\n", classification_report(y_valid.iloc[:, i], y_pred[:, i]))


Accuracy for cyber_label: 0.9404761904761905
Classification Report for cyber_label:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       235
           1       1.00      0.12      0.21        17

    accuracy                           0.94       252
   macro avg       0.97      0.56      0.59       252
weighted avg       0.94      0.94      0.92       252

Accuracy for environmental_issue: 0.8611111111111112
Classification Report for environmental_issue:
               precision    recall  f1-score   support

           0       0.86      0.98      0.92       200
           1       0.87      0.38      0.53        52

    accuracy                           0.86       252
   macro avg       0.86      0.68      0.73       252
weighted avg       0.86      0.86      0.84       252



In [7]:
# MultiOutput Classifier
multioutput_classifier = MultiOutputClassifier(LogisticRegression(random_state=42), n_jobs=-1)
multioutput_classifier.fit(X_train_usmpl, y_train_usmpl)


# Prediction and evaluation
y_pred = multioutput_classifier.predict(X_valid)
for i, label in enumerate(y_train_usmpl.columns):
    print(f"Accuracy for {label}: {accuracy_score(y_valid.iloc[:, i], y_pred[:, i])}")
    print(f"Classification Report for {label}:\n", classification_report(y_valid.iloc[:, i], y_pred[:, i]))


Accuracy for cyber_label: 0.8809523809523809
Classification Report for cyber_label:
               precision    recall  f1-score   support

           0       0.98      0.89      0.93       235
           1       0.32      0.71      0.44        17

    accuracy                           0.88       252
   macro avg       0.65      0.80      0.69       252
weighted avg       0.93      0.88      0.90       252

Accuracy for environmental_issue: 0.8412698412698413
Classification Report for environmental_issue:
               precision    recall  f1-score   support

           0       0.95      0.84      0.89       200
           1       0.58      0.85      0.69        52

    accuracy                           0.84       252
   macro avg       0.77      0.84      0.79       252
weighted avg       0.88      0.84      0.85       252



In [8]:
# MultiOutput Classifier
multioutput_classifier = MultiOutputClassifier(SVC(C=0.5, random_state=42), n_jobs=-1)
multioutput_classifier.fit(X_train_usmpl, y_train_usmpl)


# Prediction and evaluation
y_pred = multioutput_classifier.predict(X_valid)
for i, label in enumerate(y_train_usmpl.columns):
    print(f"Accuracy for {label}: {accuracy_score(y_valid.iloc[:, i], y_pred[:, i])}")
    print(f"Classification Report for {label}:\n", classification_report(y_valid.iloc[:, i], y_pred[:, i]))


Accuracy for cyber_label: 0.9166666666666666
Classification Report for cyber_label:
               precision    recall  f1-score   support

           0       0.98      0.93      0.95       235
           1       0.43      0.71      0.53        17

    accuracy                           0.92       252
   macro avg       0.70      0.82      0.74       252
weighted avg       0.94      0.92      0.93       252

Accuracy for environmental_issue: 0.8492063492063492
Classification Report for environmental_issue:
               precision    recall  f1-score   support

           0       0.94      0.86      0.90       200
           1       0.60      0.79      0.68        52

    accuracy                           0.85       252
   macro avg       0.77      0.83      0.79       252
weighted avg       0.87      0.85      0.86       252



In [9]:
# MultiOutput Classifier
multioutput_classifier = MultiOutputClassifier(SVC(C=0.8, random_state=42), n_jobs=-1)
multioutput_classifier.fit(X_train_usmpl, y_train_usmpl)


# Prediction and evaluation
y_pred = multioutput_classifier.predict(X_valid)
for i, label in enumerate(y_train_usmpl.columns):
    print(f"Accuracy for {label}: {accuracy_score(y_valid.iloc[:, i], y_pred[:, i])}")
    print(f"Classification Report for {label}:\n", classification_report(y_valid.iloc[:, i], y_pred[:, i]))


Accuracy for cyber_label: 0.9404761904761905
Classification Report for cyber_label:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97       235
           1       0.55      0.71      0.62        17

    accuracy                           0.94       252
   macro avg       0.76      0.83      0.79       252
weighted avg       0.95      0.94      0.94       252

Accuracy for environmental_issue: 0.8571428571428571
Classification Report for environmental_issue:
               precision    recall  f1-score   support

           0       0.94      0.88      0.91       200
           1       0.62      0.79      0.69        52

    accuracy                           0.86       252
   macro avg       0.78      0.83      0.80       252
weighted avg       0.87      0.86      0.86       252

